In [2]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from keras.datasets import mnist, imdb, reuters


#------------------- def methods -----------------------#

# from [[1, 3, 44], [2, 313, 87, 100], [34, 22]] -> matrix [[0, 0, 1],
#                                                          [1, 0, 0, 0], ... ]
def transformData(data, dimHorizontalAxes):
    result = np.zeros((len(data), dimHorizontalAxes))
    
    currentReview = 0
    for review in data:
        for word in review:
            if result[currentReview, word] == 0:
                # a word could be used multiple times in a single review
                result[currentReview, word] = 1
        currentReview += 1
        
    # print(result)
    return result

def transformLabels(data, dimHorizontalAxes):
    result = np.zeros((len(data), dimHorizontalAxes))
    
    currentSample = 0
    for lbl in data:
        result[currentSample, lbl] = 1
        currentSample += 1
 
    # print(result)
    return result

def check_method(tmpTensor):
    counter = 0
    for i in range(len(tmpTensor[0])):
        if tmpTensor[0, i] != tmpTensor[1, i]:
            counter += 1

    return counter
#-------------------------------------------------------#

In [3]:
# load data and analyze the type, rank and dimensionality for each rank 
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000) # at most 10000 words

print('Data types')
print('Train data and labels')
print(type(train_data))
print(type(train_labels))
print('\nTest data and labels')
print(type(test_data))
print(type(test_labels))

print('\nRank and dimensionality for each axis')
print('- train data:')
print(train_data.ndim)
print(train_data.shape)

print('- train labels:')
print(train_labels.ndim)
print(train_labels.shape)


print('\n- test data')
print(test_data.ndim)
print(test_data.shape)

print('- test labels:')
print(test_labels.ndim)
print(test_labels.shape)

Data types
Train data and labels
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

Test data and labels
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

Rank and dimensionality for each axis
- train data:
1
(8982,)
- train labels:
1
(8982,)

- test data
1
(2246,)
- test labels:
1
(2246,)


In [4]:
print('Train data content')
print(train_data[0], ' - label: ', train_labels[0])

print('\nTest data content')
print(test_data[0], ' - label: ', test_labels[0])

Train data content
[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]  - label:  3

Test data content
[1, 4, 1378, 2025, 9, 697, 4622, 111, 8, 25, 109, 29, 3650, 11, 150, 244, 364, 33, 30, 30, 1398, 333, 6, 2, 159, 9, 1084, 363, 13, 2, 71, 9, 2, 71, 117, 4, 225, 78, 206, 10, 9, 1214, 8, 4, 270, 5, 2, 7, 748, 48, 9, 2, 7, 207, 1451, 966, 1864, 793, 97, 133, 336, 7, 4, 493, 98, 273, 104, 284, 25, 39, 338, 22, 905, 220, 3465, 644, 59, 20, 6, 119, 61, 11, 15, 58, 579, 26, 10, 67, 7, 4, 738, 98, 43, 88, 333, 722, 12, 20, 6, 19, 746, 35, 15, 10, 9, 1214, 855, 129, 783, 21, 4, 2280, 244, 364, 51, 16, 299, 452, 16, 515, 4, 99, 29, 5, 4, 364, 281, 48, 10, 9, 1214, 23, 644, 47, 20, 324, 27, 56

In [5]:
# we have to split train_data in x_train and x_val with their labels

#AND

# we must transform train_data in a tensor, "1 value in each cell of the new data structure" IS THE RULE
# the new data structure for our problem is a tensor with rank 2
# the features inside the matrix are the words, represented by a 0/1 for a word on the horizontal axes

dimensionalityHorizontalAxes = 10000 # we have almost 10000 words
x_train = transformData(train_data[:5982], dimensionalityHorizontalAxes)
x_val = transformData(train_data[5982:len(train_data)], dimensionalityHorizontalAxes)

x_test = transformData(test_data, dimensionalityHorizontalAxes)

In [6]:
# in a multiclass classification problem the best format for the labels is a tensor of rank 2
# for each sample on the vertical axes, and a vector on the horizontal axes with 0/1 for the class of that sample
# the dimensionality for the horizontal axes is equalt to the number of class labels
# 0 0 1 0 0 0 ... 0 0 0
# 1 0 0 0 0 0 ... 0 0 0
# 0 0 0 0 0 0 ... 1 0 0

cardinalityClassLabels = 46
y_train = transformLabels(train_labels[:5982], cardinalityClassLabels)
y_val = transformLabels(train_labels[5982:len(train_labels)], cardinalityClassLabels)
y_test = transformLabels(test_labels, cardinalityClassLabels)

In [7]:
print('New data types')
print('Train data and labels')
print(type(x_train))
print(type(y_train))
print('\nValidation data and labels')
print(type(x_val))
print(type(y_val))
print('\nTest data and labels')
print(type(x_test))
print(type(y_test))

print('\nRank and dimensionality for each axis')
print('- train data:')
print(x_train.ndim)
print(x_train.shape)

print('- train labels:')
print(y_train.ndim)
print(y_train.shape)



print('- validation data:')
print(x_val.ndim)
print(x_val.shape)

print('- validation labels:')
print(y_val.ndim)
print(y_val.shape)



print('\n- test data')
print(x_test.ndim)
print(x_test.shape)

print('- test labels:')
print(y_test.ndim)
print(y_test.shape)

New data types
Train data and labels
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

Validation data and labels
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

Test data and labels
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

Rank and dimensionality for each axis
- train data:
2
(5982, 10000)
- train labels:
2
(5982, 46)
- validation data:
2
(3000, 10000)
- validation labels:
2
(3000, 46)

- test data
2
(2246, 10000)
- test labels:
2
(2246, 46)


In [8]:
print('\nTrain data content')
print(x_train[0], ' - label: ', train_labels[0])

print('\nTest data content')
print(x_test[0], ' - label: ', test_labels[0])


Train data content
[0. 1. 1. ... 0. 0. 0.]  - label:  3

Test data content
[0. 1. 1. ... 0. 0. 0.]  - label:  3


In [9]:
# check method
print(check_method(x_train))
print(check_method(x_test))
print(check_method(x_val))

70
326
104


In [12]:
# we an build our NN considering the problem we are going to face

# model definition 
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,))) # 10000 features == words index
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))
          
          
from keras import optimizers
          
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', 
              metrics=['accuracy']) 

# use verbose=0 if you don't want to see keras text
model.fit(x_train, y_train, epochs=4, batch_size=512, validation_data=(x_val, y_val))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 5982 samples, validate on 3000 samples
Epoch 1/4
5982/5982 [==============================] - 4s 749us/step - loss: 2.6911 - accuracy: 0.5000 - val_loss: 1.8369 - val_accuracy: 0.6553
Epoch 2/4
5982/5982 [==============================] - 2s 407us/step - loss: 1.5294 - accuracy: 0.6876 - val_loss: 1.4045 - val_accuracy: 0.7037
Epoch 3/4
5982/5982 [==============================] - 2s 409us/step - loss: 1.1353 - accuracy: 0.7589 - val_loss: 1.2212 - val_accuracy: 0.7407
Epoch 4/4
5982/5982 [==============================] - 2s 406us/step - loss: 0.9054 - accuracy: 0.8138 - val_loss: 1.1131 - val_accuracy: 0.7670


In [13]:
print(model.evaluate(x_test, y_test))

2246/2246 [==============================] - 1s 382us/step
[1.1791188423907448, 0.7528940439224243]


In [ ]:
# plot loss score for train and validation data to understand where the overfitting happens
